In [0]:
%pip install scikit-learn==1.5.0 xgboost==1.7.2

Python interpreter will be restarted.
  Attempting uninstall: threadpoolctl
    Found existing installation: threadpoolctl 2.2.0
    Not uninstalling threadpoolctl at /databricks/python3/lib/python3.9/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-aa3fec72-595f-4797-b90a-b3b69f60f8ab
    Can't uninstall 'threadpoolctl'. No files were found to uninstall.
  Attempting uninstall: joblib
    Found existing installation: joblib 1.1.1
    Not uninstalling joblib at /databricks/python3/lib/python3.9/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-aa3fec72-595f-4797-b90a-b3b69f60f8ab
    Can't uninstall 'joblib'. No files were found to uninstall.
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.0.2
    Not uninstalling scikit-learn at /databricks/python3/lib/python3.9/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-aa3fec72-595f-4797-b90a-b3b69f60f8ab
    Can't uninsta

In [0]:
import pyspark.sql.functions as psf
from pyspark.sql.types import DoubleType, FloatType
from pyspark.ml import PipelineModel
import joblib
import pandas as pd
from pyspark.sql import Window

import re
from datetime import datetime

In [0]:
def find_latest_snapshot_table(spark, database_name, table_pattern):
    """
    Find the latest snapshot table based on the date appended to the table name.

    :param spark: An active Spark session.
    :param database_name: The name of the database where the tables are stored.
    :param table_pattern: The base pattern of the table name before the date.
    :return: The full table name of the latest snapshot.
    """
    # Use Spark SQL to list all tables in the specified database
    tables_df = spark.sql(f"SHOW TABLES IN {database_name}")

    # Filter tables based on the table pattern and extract table names
    pattern = re.escape(table_pattern) + r"_(\d{4}_\d{2}_\d{2})$"  # Date format is 'YYYY_MM_DD'
    matched_tables = [row.tableName for row in tables_df.collect() if re.match(pattern, row.tableName)]

    # Extract dates from table names and find the latest one
    dates = [re.search(r"_(\d{4}_\d{2}_\d{2})$", table_name).group(1) for table_name in matched_tables]
    latest_date = max(dates, key=lambda d: datetime.strptime(d, '%Y_%m_%d'))

    # Construct the full table name of the latest snapshot
    latest_table_name = f"{table_pattern}_{latest_date}"

    # Return the full table name of the latest snapshot
    return f"{database_name}.{latest_table_name}"


In [0]:

# database_name = "hive_metastore.smarty_port_out_churner"
# table_pattern = "inference_data"

# # Find the latest snapshot table name
# latest_snapshot_table_name = find_latest_snapshot_table(spark, database_name, table_pattern)


In [0]:
database_name = "hive_metastore.smarty_inactivity_churner"
table_pattern = "inference_data"

INFERENCE_DATA_PATH = find_latest_snapshot_table(spark, database_name, table_pattern)
MODEL_PATH = '/dbfs/dbfs/FileStore/shared_uploads/frehmani@corpuk.net/smarty/inactivity_churner_14_v2_model.joblib'

RAW_OUTPUT_TABLE = '/dbfs/FileStore/shared_uploads/frehmani@corpuk.net/smarty/inactivity_churner_v1_predictions'

SNOWFLAKE_OUTPUT_TABLE = 'FR_MODEL_SCORES'
MODEL_ID = 'INACTIVITY_CHURNER-V1'

# Get secrets for accessing Snowflake (assuming dbutils is available)
prod_user = dbutils.secrets.get(scope="SMARTY", key="smarty_sc_u")
prod_pwd = dbutils.secrets.get(scope="SMARTY", key="smarty_sc_p")

# Snowflake options
options = {
    "sfUrl": 'threemobile.west-europe.privatelink.snowflakecomputing.com',
    "sfUser": prod_user,
    "sfPassword": prod_pwd,
    "sfDatabase": 'PROD_SMARTY',
    "sfSchema": 'DEFAULT',
    "sfWarehouse": 'PROD_SMARTY_WH',
    "sfRole": 'SF_PROD_SMARTY'
}

In [0]:
features_to_include = [
    'PlanTextUKCount_Last6Month',
     'PlanDataUKGB_Last6Month',
     'TotalPaymentRetryAttempt_Last6Month',
     'PlanDataUKGB_LastWeek', 
     'Minute_Month_6Month_Ratio', 
     'Data_Month_6Month_Ratio', 
     'AgeRange', 
     'LastRenewalNumber', 
     'PlanVoiceUKCount_Last6Month',
     'PlanVoiceUKMinute_LastWeek', 
     'Days_Since_LastRenewalDate', 
     'Channel',
     'Data_Week_Month_Ratio', 
     'Minute_Week_Month_Ratio'
     ]

data_types ={
    'PlanTextUKCount_Last6Month': 'Numerical',
    'PlanDataUKGB_Last6Month': 'Numerical',
    'TotalPaymentRetryAttempt_Last6Month': 'Numerical',
    'PlanDataUKGB_LastWeek': 'Numerical',
    'Minute_Month_6Month_Ratio': 'Numerical',
    'Data_Month_6Month_Ratio': 'Numerical',
    'LastRenewalNumber': 'Numerical',
    'PlanVoiceUKCount_Last6Month': 'Numerical',
    'PlanVoiceUKMinute_LastWeek': 'Numerical',
    'Days_Since_LastRenewalDate': 'Numerical',
    'Data_Week_Month_Ratio': 'Numerical',
    'Minute_Week_Month_Ratio': 'Numerical',
    'AgeRange': 'Categorical',
    'Channel': 'Categorical'
}

fill_with_zero_columns = [
     'PlanTextUKCount_Last6Month',
     'PlanDataUKGB_Last6Month',
     'TotalPaymentRetryAttempt_Last6Month',
     'PlanDataUKGB_LastWeek', 
     'Minute_Month_6Month_Ratio', 
     'Data_Month_6Month_Ratio', 
     'LastRenewalNumber', 
     'PlanVoiceUKCount_Last6Month',
     'PlanVoiceUKMinute_LastWeek', 
     'Days_Since_LastRenewalDate', 
     'Data_Week_Month_Ratio', 
     'Minute_Week_Month_Ratio'
     ]

In [0]:

# features_to_include = [
#     'PlanTextUKCount_Last6Month',
#      'PlanDataUKGB_Last6Month',
#      'TotalPaymentRetryAttempt_Last6Month',
#      'PlanDataUKGB_LastWeek', 
#      'Minute_Month_6Month_Ratio', 
#      'Data_Month_6Month_Ratio', 
#      'AgeRange', 
#      'LastRenewalNumber', 
#      'PlanVoiceUKCount_Last6Month',
#      'PlanVoiceUKMinute_LastWeek', 
#      'Days_Since_LastRenewalDate', 
#      'Channel',
#      'Data_Week_Month_Ratio', 
#      'Minute_Week_Month_Ratio'
#      ]

# data_types ={
# 'MarketingEmailOpen_LastWeek': 'Numerical',
#  'PaymentSuccessfullyValue_LastMonth': 'Numerical',
#  'ReferralCountVoucherVariable': 'Numerical',
#  'EUFairUsageAddonSoldDaysFor': 'Numerical',
#  'PaymentSuccessfullyCountAuto_Last6Month': 'Numerical',
#  #'LastPlanChangeAt': 'Unknown',
#  'LastPlan': 'Categorical',
#  'Renewal3NetDataUsagePct': 'Numerical',
#  'Renewal2PlanSMSUsage': 'Numerical',
#  'MarketingAppClicked_LastWeek': 'Numerical',
#  'AccountBalanceOwed': 'Numerical',
#  'ReferralCountFreeMonth_LastMonth': 'Numerical',
#  'ReferralCountVoucherVariable_LastWeek': 'Numerical',
#  'PaymentSuccessfullyCountManual_LastMonth': 'Numerical',
#  #'AutoRenewChangeSource': 'Categorical',
#  'ComplaintTicketOpen_LastWeek': 'Numerical',
#  #'AutoRenew': 'Boolean',
#  'PaymentSuccessfullyValueAuto_LastMonth': 'Numerical',
#  #'Days_Since_AccountStatusChangedAt': 'Numerical',
#  'ReferralCountVoucherVariable_Last6Month': 'Numerical',
#  'ReferralCountVoucher': 'Numerical',
#  'DataAddonSoldAllowance': 'Numerical',
#  'Event_Occurred_LastPlanChangeAt': 'Numerical',
#  'PaymentFailureCount': 'Numerical',
#  'NextRenewalPlanSmsAllowance': 'Numerical',
#  'PaymentFailureCountOther_LastMonth': 'Numerical',
#  'PaymentFailureCountOther_Last6Month': 'Numerical',
#  'WeeksSinceActivtion': 'Numerical',
#  #'Age': 'Numerical',
#   'AgeRange': 'Categorical',
#  'CashAddonSoldCost': 'Numerical',
#  'PaymentSuccessfullyCountManual_LastWeek': 'Numerical',
#  'Minute_Month_6Month_PctChange': 'Numerical',
#  'PlanVoiceUKCount_Last6Month': 'Numerical',
#  'LastRenewalPromoFixedValue': 'Numerical',
#  'MarketingSMSOpen_LastWeek': 'Numerical',
#  'NextRenewalPlan': 'Categorical',
#  'Days_Since_LostSimAt': 'Numerical',
#  'WebChatEmailMatched_Last6Month': 'Numerical',
#  'CurrentPlanPrice': 'Numerical',
#  'PlanVoiceUKMinute_Last6Month': 'Numerical',
#  'MarketingSMSSent_LastMonth': 'Numerical',
#  'AvgInactivityDuration': 'Numerical',
#  'Renewal1PromoDataUsage': 'Numerical',
#  'Renewal2PromoDataUsage': 'Numerical',
#  'LastRenewalNumber': 'Numerical',
#  'PlanVoiceUKMinute_LastWeek': 'Numerical',
#  #'PaymentSuccessfullyCountAuto': 'Numerical',
#  'PaymentSuccessfullyValueManual_LastMonth': 'Numerical',
#  'Renewal3PlanSMSUsage': 'Numerical',
#  'OtherDataUKGB_Last6Month': 'Numerical',
#  'LastRenewalDiscountedPlanCost': 'Numerical',
#  #'PaymentSuccessfullyValue': 'Numerical',
#  'Renewal3PlanSMSUsagePct': 'Numerical',
#  'FailedCDROtherCount_LastWeek': 'Numerical',
#  'FailedCDROtherCount_LastMonth': 'Numerical',
#  'FailedCDRNotEnoughFundsCount_LastMonth': 'Numerical',
#  'PaymentSuccessfullyValue_Last6Month': 'Numerical',
#  'PaymentSuccessfullyCount': 'Numerical',
#  'LastRenewalAccountBalanceUsed': 'Numerical',
#  'PaymentFailureCountInsufficientFunds_Last6Month': 'Numerical',
#  'Renewal2PlanVoiceUsagePct': 'Numerical',
#  'NextRenewalPlanType': 'Categorical',
#  'PaymentSuccessfullyCountAuto_LastMonth': 'Numerical',
#  'ReferralAvgMRC_Last6Month': 'Numerical',
#  'CashAddonSoldCount': 'Numerical',
#  'PaymentSuccessfullyCountManual': 'Numerical',
#  'PaymentSuccessfullyValueManual': 'Numerical',
#  'PlanChangeCount_Last6Month': 'Numerical',
#  'ReferralCountVoucher_LastWeek': 'Numerical',
#  'ReferralAvgMRC_LastMonth': 'Numerical',
#  'Renewal1PlanDataUsage': 'Numerical',
#  'Data_Week_Month_PctChange': 'Numerical',
#  'Days_Since_LastRenewalDate': 'Numerical',
#  'Renewal1PlanSMSUsagePct': 'Numerical',
#  #'PaymentSuccessfullyValueAuto': 'Numerical',
#  'GroupSize': 'Numerical',
#  'FailedCDRAccountStatusBlockedCount_LastMonth': 'Numerical',
#  'MarketingEmailOpen_Last6Month': 'Numerical',
#  'PaymentFailureCountInsufficientFunds_LastWeek': 'Numerical',
#  'Data_Week_Month_Ratio': 'Numerical',
#  'MarketingAppConverted_LastWeek': 'Numerical',
#  'Text_Week_Month_Ratio': 'Numerical',
#  'MarketingEmailSent_Last6Month': 'Numerical',
#  'NextRenewalPlanDataAllowance': 'Numerical',
#  'MarketingEmailClicked_LastWeek': 'Numerical',
#  'Text_Month_6Month_AbsChange': 'Numerical',
#  'Segment': 'Categorical',
#  'Event_Occurred_LostSimAt': 'Numerical',
#  'MarketingEmailDelivered_Last6Month': 'Numerical',
#  'PaymentSuccessfullyValueAuto_Last6Month': 'Numerical',
#  'Minute_Week_Month_Ratio': 'Numerical',
#  'RenewalCount': 'Numerical',
#  'LastPlanChangeDirectionByMRC': 'Categorical',
#  'Event_Occurred_InfoRequestAt': 'Numerical',
#  'PlanTextUKCount_Last6Month': 'Numerical',
#  'PaymentFailureCountInsufficientFunds': 'Numerical',
#  'LastRenewalPlanType': 'Categorical',
#  'PaymentFailureCountInsufficientFunds_LastMonth': 'Numerical',
#  'PlanChangeCount': 'Numerical',
#  'PaymentSuccessfullyValueManual_Last6Month': 'Numerical',
#  'PlanChangeCount_LastMonth': 'Numerical',
#  'FailedCDROtherCount_Last6Month': 'Numerical',
#  'PaymentSuccessfullyValue_LastWeek': 'Numerical',
#  'PaymentSuccessfullyCount_LastWeek': 'Numerical',
#  'MarketingEmailConverted_LastWeek': 'Numerical',
#  'PaymentFailureCount_Last6Month': 'Numerical',
#  'LastPlanPrice': 'Numerical',
#  'PaymentFailureCount_LastWeek': 'Numerical',
#  'ReferralCountFreeMonth_Last6Month': 'Numerical',
#  'ReferralCountVoucherVariable_LastMonth': 'Numerical',
#  'LastPlanDataAllowance': 'Numerical',
#  'Renewal2NetDataUsagePct': 'Numerical',
#  'WasReferredRewardType': 'Categorical',
#  'EUFairUsageAddonSoldCount': 'Numerical',
#  'FailedCDRNotEnoughFundsCount_Last6Month': 'Numerical',
#  'Minute_Week_Month_AbsChange': 'Numerical',
#  'Event_Occurred_LastRenewalDate': 'Numerical',
#  'InfoRequest_LastWeek': 'Numerical',
#  'EUFairUsageAddonSoldCost': 'Numerical',
#  'LastRenewalPlanDataAllowance': 'Numerical',
#  'PaymentSuccessfullyValueAuto_LastWeek': 'Numerical',
#  'Event_Occurred_AutoRenewChangeDate': 'Numerical',
#  'PaymentSuccessfullyCountAuto_LastWeek': 'Numerical',
#  'ReplacementSimOrder_LastWeek': 'Numerical',
#  'DataAddonSoldCount': 'Numerical',
#  'PaymentRefundCount_Last6Month': 'Numerical',
#  'LastPlanChangeDaySince': 'Numerical',
#  'MarketingEmailDelivered_LastMonth': 'Numerical',
#  'ReferralCountFreeMonth_LastWeek': 'Numerical',
#  'Days_Since_GroupJoinedAt': 'Numerical',
#  'MarketingSMSSent_Last6Month': 'Numerical',
#  'InactivityPaymentFailureReason': 'Categorical',
#  'Data_Month_6Month_AbsChange': 'Numerical',
#  'ChannelGroup': 'Categorical',
#  'Text_Month_6Month_PctChange': 'Numerical',
#  'InactivityAccidentalInstanceCount': 'Numerical',
#  'PaymentSuccessfullyValueManual_LastWeek': 'Numerical',
#  'Event_Occurred_AccountStatusChangedAt': 'Numerical',
#  'Renewal1PlanVoiceUsage': 'Numerical',
#  'Renewal2PlanSMSUsagePct': 'Numerical',
#  'UkUnlimitedAddonSoldCost': 'Numerical',
#  'ReferralAvgMRC_LastWeek': 'Numerical',
#  'DaysInactive': 'Numerical',
#  'PaymentFailureCountOther_LastWeek': 'Numerical',
#  'CurrentPlanDataAllowance': 'Numerical',
#  'DataAddonSoldCost': 'Numerical',
#  'Renewal1PlanDataUsagePct': 'Numerical',
#  'ReferralCountVoucher_LastMonth': 'Numerical',
#  'AvgMRC': 'Numerical',
#  'PlanDataUKGB_LastWeek': 'Numerical',
#  'ReferralCount': 'Numerical',
#  'PaymentSuccessfullyCountManual_Last6Month': 'Numerical',
#  'PlanDataUKGB_Last6Month': 'Numerical',
#  'Minute_Month_6Month_Ratio': 'Numerical',
#  'TariffChangeOrder_LastWeek': 'Numerical',
#  'UkUnlimitedAddonSoldDaysFor': 'Numerical',
#  'Tenure': 'Numerical',
#  'Renewal2PromoDataUsagePct': 'Numerical',
#  'Renewal1NetDataUsagePct': 'Numerical',
#  'MarketingAppOpen_LastWeek': 'Numerical',
#  'Renewal3PromoDataUsage': 'Numerical',
#  'PaymentRefundCount_LastWeek': 'Numerical',
#  'Text_Month_6Month_Ratio': 'Numerical',
#  'NumberSwapOrder_LastWeek': 'Numerical',
#  'PlanTextUKCount_LastMonth': 'Numerical',
#  'TicketOpen_LastWeek': 'Numerical',
#  'MarketingSMSConverted_LastWeek': 'Numerical',
#  'Days_Since_InfoRequestAt': 'Numerical',
#  #'PortedInFrom': 'Categorical',
#  'ComplaintAgeOfOldestOpenTicket_LastWeek': 'Numerical',
#  'NextRenewalPlanCost': 'Numerical',
#  'Data_Month_6Month_Ratio': 'Numerical',
#  'Text_Week_Month_AbsChange': 'Numerical',
#  'ReferralCountVoucher_Last6Month': 'Numerical',
#  'PaymentRefundCount': 'Numerical',
#  'Renewal2PlanDataUsagePct': 'Numerical',
#  'PaymentFailureCountOther': 'Numerical',
#  'MarketingSMSDelivered_Last6Month': 'Numerical',
#  'Text_Week_Month_PctChange': 'Numerical',
#  'UkUnlimitedAddonSoldCount': 'Numerical',
#  'Renewal3PromoDataUsagePct': 'Numerical',
#  'PaymentRefundCount_LastMonth': 'Numerical',
#  'LastRenewalPlan': 'Categorical',
#  'Days_Since_LastPlanChangeAt': 'Numerical',
#  'PlanVoiceUKCount_LastMonth': 'Numerical',
#  'InactivityReason': 'Categorical',
#  'MarketingSMSClicked_LastWeek': 'Numerical',
#  'STACRequest_Last6Month': 'Numerical',
#  'MonthsSinceActivtion': 'Numerical',
#  'Renewal2PlanVoiceUsage': 'Numerical',
#  'PlanChangeCount_LastWeek': 'Numerical',
#  'PaymentFailureCount_LastMonth': 'Numerical',
#  'Renewal3PlanDataUsagePct': 'Numerical',
#  'LastComplaintReason': 'Categorical',
#  'PlanVoiceUKCount_LastWeek': 'Numerical',
#  'SimSwapOrder_LastWeek': 'Numerical',
#  'ComplaintTicketRaised_LastWeek': 'Numerical',
#  'PaymentSuccessfullyCount_LastMonth': 'Numerical',
#  'CurrentPlan': 'Categorical',
#  'FailedCDRAccountStatusBlockedCount_LastWeek': 'Numerical',
#  'ReferralCountFreeMonth': 'Numerical',
#  'ReferralAvgMRC': 'Numerical',
#  'LastPlanChangeDirectionByDataAllowance': 'Categorical',
#  'PlanDataUKGB_LastMonth': 'Numerical',
#  'FailedCDRAccountStatusBlockedCount_Last6Month': 'Numerical',
#  'Data_Week_Month_AbsChange': 'Numerical',
#  'Renewal1PromoDataUsagePct': 'Numerical',
#  'OtherDataUKGB_LastWeek': 'Numerical',
#  'PlanTextUKCount_LastWeek': 'Numerical',
#  'FailedCDRNotEnoughFundsCount_LastWeek': 'Numerical',
#  'Renewal2PlanDataUsage': 'Numerical',
#  'MarketingEmailSent_LastMonth': 'Numerical',
#  'InactivityAutoRenewStatus': 'Boolean',
#  'LastRenewalPlanCost': 'Numerical',
#  'NextRenewalGroupDiscountPercentage': 'Numerical',
#  'NextRenewalAccountBalanceUsed': 'Numerical',
#  'Minute_Week_Month_PctChange': 'Numerical',
#  'PaymentSuccessfullyCount_Last6Month': 'Numerical',
#  'NextRenewalPlanVoiceAllowance': 'Numerical',
#  'Renewal1PlanVoiceUsagePct': 'Numerical',
#  'Renewal3PlanVoiceUsage': 'Numerical',
#  'Renewal1PlanSMSUsage': 'Numerical',
#  'Event_Occurred_GroupJoinedAt': 'Numerical',
#  'InactivityInstanceCount': 'Numerical',
#  'Renewal3PlanVoiceUsagePct': 'Numerical',
#  'Days_Since_AutoRenewChangeDate': 'Numerical',
#  'TicketRaised_LastWeek': 'Numerical',
#  'Minute_Month_6Month_AbsChange': 'Numerical',
#  'Channel': 'Categorical',
#  'Renewal3PlanDataUsage': 'Numerical',
#  'LastRenewalPaymentTaken': 'Numerical',
#  'TotalPaymentRetryAttempt_Last6Month': 'Numerical',
#  'Data_Month_6Month_PctChange': 'Numerical',
#  'LeadGeneratorGroup': 'Categorical',
#  'PlanVoiceUKMinute_LastMonth': 'Numerical',
#  'OtherDataUKGB_LastMonth': 'Numerical'}


# fill_with_zero_columns = [
#  'MarketingEmailOpen_LastWeek',
#  'PaymentSuccessfullyValue_LastMonth',
#  'ReferralCountVoucherVariable',
#  'EUFairUsageAddonSoldDaysFor',
#  'PaymentSuccessfullyCountAuto_Last6Month',
#  'Renewal3NetDataUsagePct',
#  'Renewal2PlanSMSUsage',
#  'MarketingAppClicked_LastWeek',
#  'AccountBalanceOwed',
#  'ReferralCountFreeMonth_LastMonth',
#  'ReferralCountVoucherVariable_LastWeek',
#  'PaymentSuccessfullyCountManual_LastMonth',
#  'ComplaintTicketOpen_LastWeek',
#  'PaymentSuccessfullyValueAuto_LastMonth',
#  #'Days_Since_AccountStatusChangedAt',
#  'ReferralCountVoucherVariable_Last6Month',
#  'ReferralCountVoucher',
#  'DataAddonSoldAllowance',
#  'Event_Occurred_LastPlanChangeAt',
#  'PaymentFailureCount',
#  'NextRenewalPlanSmsAllowance',
#  'PaymentFailureCountOther_LastMonth',
#  'PaymentFailureCountOther_Last6Month',
#  'WeeksSinceActivtion',
#  #'Age',
#  'CashAddonSoldCost',
#  'PaymentSuccessfullyCountManual_LastWeek',
#  'Minute_Month_6Month_PctChange',
#  'PlanVoiceUKCount_Last6Month',
#  'LastRenewalPromoFixedValue',
#  'MarketingSMSOpen_LastWeek',
#  'Days_Since_LostSimAt',
#  'WebChatEmailMatched_Last6Month',
#  'CurrentPlanPrice',
#  'PlanVoiceUKMinute_Last6Month',
#  'MarketingSMSSent_LastMonth',
#  'AvgInactivityDuration',
#  'Renewal1PromoDataUsage',
#  'Renewal2PromoDataUsage',
#  'LastRenewalNumber',
#  'PlanVoiceUKMinute_LastWeek',
#  #'PaymentSuccessfullyCountAuto',
#  'PaymentSuccessfullyValueManual_LastMonth',
#  'Renewal3PlanSMSUsage',
#  'OtherDataUKGB_Last6Month',
#  'LastRenewalDiscountedPlanCost',
#  #'PaymentSuccessfullyValue',
#  'Renewal3PlanSMSUsagePct',
#  'FailedCDROtherCount_LastWeek',
#  'FailedCDROtherCount_LastMonth',
#  'FailedCDRNotEnoughFundsCount_LastMonth',
#  'PaymentSuccessfullyValue_Last6Month',
#  'PaymentSuccessfullyCount',
#  'LastRenewalAccountBalanceUsed',
#  'PaymentFailureCountInsufficientFunds_Last6Month',
#  'Renewal2PlanVoiceUsagePct',
#  'PaymentSuccessfullyCountAuto_LastMonth',
#  'ReferralAvgMRC_Last6Month',
#  'CashAddonSoldCount',
#  'PaymentSuccessfullyCountManual',
#  'PaymentSuccessfullyValueManual',
#  'PlanChangeCount_Last6Month',
#  'ReferralCountVoucher_LastWeek',
#  'ReferralAvgMRC_LastMonth',
#  'Renewal1PlanDataUsage',
#  'Data_Week_Month_PctChange',
#  'Days_Since_LastRenewalDate',
#  'Renewal1PlanSMSUsagePct',
#  #'PaymentSuccessfullyValueAuto',
#  'GroupSize',
#  'FailedCDRAccountStatusBlockedCount_LastMonth',
#  'MarketingEmailOpen_Last6Month',
#  'PaymentFailureCountInsufficientFunds_LastWeek',
#  'Data_Week_Month_Ratio',
#  'MarketingAppConverted_LastWeek',
#  'Text_Week_Month_Ratio',
#  'MarketingEmailSent_Last6Month',
#  'NextRenewalPlanDataAllowance',
#  'MarketingEmailClicked_LastWeek',
#  'Text_Month_6Month_AbsChange',
#  'Event_Occurred_LostSimAt',
#  'MarketingEmailDelivered_Last6Month',
#  'PaymentSuccessfullyValueAuto_Last6Month',
#  'Minute_Week_Month_Ratio',
#  'RenewalCount',
#  'Event_Occurred_InfoRequestAt',
#  'PlanTextUKCount_Last6Month',
#  'PaymentFailureCountInsufficientFunds',
#  'PaymentFailureCountInsufficientFunds_LastMonth',
#  'PlanChangeCount',
#  'PaymentSuccessfullyValueManual_Last6Month',
#  'PlanChangeCount_LastMonth',
#  'FailedCDROtherCount_Last6Month',
#  'PaymentSuccessfullyValue_LastWeek',
#  'PaymentSuccessfullyCount_LastWeek',
#  'MarketingEmailConverted_LastWeek',
#  'PaymentFailureCount_Last6Month',
#  'LastPlanPrice',
#  'PaymentFailureCount_LastWeek',
#  'ReferralCountFreeMonth_Last6Month',
#  'ReferralCountVoucherVariable_LastMonth',
#  'LastPlanDataAllowance',
#  'Renewal2NetDataUsagePct',
#  'EUFairUsageAddonSoldCount',
#  'FailedCDRNotEnoughFundsCount_Last6Month',
#  'Minute_Week_Month_AbsChange',
#  'Event_Occurred_LastRenewalDate',
#  'InfoRequest_LastWeek',
#  'EUFairUsageAddonSoldCost',
#  'LastRenewalPlanDataAllowance',
#  'PaymentSuccessfullyValueAuto_LastWeek',
#  'Event_Occurred_AutoRenewChangeDate',
#  'PaymentSuccessfullyCountAuto_LastWeek',
#  'ReplacementSimOrder_LastWeek',
#  'DataAddonSoldCount',
#  'PaymentRefundCount_Last6Month',
#  'LastPlanChangeDaySince',
#  'MarketingEmailDelivered_LastMonth',
#  'ReferralCountFreeMonth_LastWeek',
#  'Days_Since_GroupJoinedAt',
#  'MarketingSMSSent_Last6Month',
#  'Data_Month_6Month_AbsChange',
#  'Text_Month_6Month_PctChange',
#  'InactivityAccidentalInstanceCount',
#  'PaymentSuccessfullyValueManual_LastWeek',
#  'Event_Occurred_AccountStatusChangedAt',
#  'Renewal1PlanVoiceUsage',
#  'Renewal2PlanSMSUsagePct',
#  'UkUnlimitedAddonSoldCost',
#  'ReferralAvgMRC_LastWeek',
#  'DaysInactive',
#  'PaymentFailureCountOther_LastWeek',
#  'CurrentPlanDataAllowance',
#  'DataAddonSoldCost',
#  'Renewal1PlanDataUsagePct',
#  'ReferralCountVoucher_LastMonth',
#  'AvgMRC',
#  'PlanDataUKGB_LastWeek',
#  'ReferralCount',
#  'PaymentSuccessfullyCountManual_Last6Month',
#  'PlanDataUKGB_Last6Month',
#  'Minute_Month_6Month_Ratio',
#  'TariffChangeOrder_LastWeek',
#  'UkUnlimitedAddonSoldDaysFor',
#  'Tenure',
#  'Renewal2PromoDataUsagePct',
#  'Renewal1NetDataUsagePct',
#  'MarketingAppOpen_LastWeek',
#  'Renewal3PromoDataUsage',
#  'PaymentRefundCount_LastWeek',
#  'Text_Month_6Month_Ratio',
#  'NumberSwapOrder_LastWeek',
#  'PlanTextUKCount_LastMonth',
#  'TicketOpen_LastWeek',
#  'MarketingSMSConverted_LastWeek',
#  'Days_Since_InfoRequestAt',
#  'ComplaintAgeOfOldestOpenTicket_LastWeek',
#  'NextRenewalPlanCost',
#  'Data_Month_6Month_Ratio',
#  'Text_Week_Month_AbsChange',
#  'ReferralCountVoucher_Last6Month',
#  'PaymentRefundCount',
#  'Renewal2PlanDataUsagePct',
#  'PaymentFailureCountOther',
#  'MarketingSMSDelivered_Last6Month',
#  'Text_Week_Month_PctChange',
#  'UkUnlimitedAddonSoldCount',
#  'Renewal3PromoDataUsagePct',
#  'PaymentRefundCount_LastMonth',
#  'Days_Since_LastPlanChangeAt',
#  'PlanVoiceUKCount_LastMonth',
#  'MarketingSMSClicked_LastWeek',
#  'STACRequest_Last6Month',
#  'MonthsSinceActivtion',
#  'Renewal2PlanVoiceUsage',
#  'PlanChangeCount_LastWeek',
#  'PaymentFailureCount_LastMonth',
#  'Renewal3PlanDataUsagePct',
#  'PlanVoiceUKCount_LastWeek',
#  'SimSwapOrder_LastWeek',
#  'ComplaintTicketRaised_LastWeek',
#  'PaymentSuccessfullyCount_LastMonth',
#  'FailedCDRAccountStatusBlockedCount_LastWeek',
#  'ReferralCountFreeMonth',
#  'ReferralAvgMRC',
#  'PlanDataUKGB_LastMonth',
#  'FailedCDRAccountStatusBlockedCount_Last6Month',
#  'Data_Week_Month_AbsChange',
#  'Renewal1PromoDataUsagePct',
#  'OtherDataUKGB_LastWeek',
#  'PlanTextUKCount_LastWeek',
#  'FailedCDRNotEnoughFundsCount_LastWeek',
#  'Renewal2PlanDataUsage',
#  'MarketingEmailSent_LastMonth',
#  'LastRenewalPlanCost',
#  'NextRenewalGroupDiscountPercentage',
#  'NextRenewalAccountBalanceUsed',
#  'Minute_Week_Month_PctChange',
#  'PaymentSuccessfullyCount_Last6Month',
#  'NextRenewalPlanVoiceAllowance',
#  'Renewal1PlanVoiceUsagePct',
#  'Renewal3PlanVoiceUsage',
#  'Renewal1PlanSMSUsage',
#  'Event_Occurred_GroupJoinedAt',
#  'InactivityInstanceCount',
#  'Renewal3PlanVoiceUsagePct',
#  'Days_Since_AutoRenewChangeDate',
#  'TicketRaised_LastWeek',
#  'Minute_Month_6Month_AbsChange',
#  'Renewal3PlanDataUsage',
#  'LastRenewalPaymentTaken',
#  'TotalPaymentRetryAttempt_Last6Month',
#  'Data_Month_6Month_PctChange',
#  'PlanVoiceUKMinute_LastMonth',
#  'OtherDataUKGB_LastMonth']



In [0]:
inference_sdf = (
    spark.read.table(INFERENCE_DATA_PATH)
    .withColumn("MODEL_END_DATE", psf.date_add(psf.col("SnapshotDate"), 20))
)

In [0]:
# Select only the features required by the model
inference_sdf = inference_sdf.select(*features_to_include,"Account_Id" ,"MODEL_END_DATE")

In [0]:
BOOL_COLS = [i for i in features_to_include if data_types[i] == "Boolean"]

NUMERICAL_COLS = [i for i in features_to_include if data_types[i] == "Numerical"]

CATEGORICAL_COLS = [
    i for i in features_to_include if data_types[i] == "Categorical"
]

inference_sdf_filled = inference_sdf.fillna(0, subset=fill_with_zero_columns)

In [0]:
inference_pdf = inference_sdf_filled.toPandas()

# Preprocess boolean columns
for col_name in BOOL_COLS:
    inference_pdf[col_name] = inference_pdf[col_name].astype(str)

# Preprocess numerical columns
for col_name in NUMERICAL_COLS:
    inference_pdf[col_name] = pd.to_numeric(inference_pdf[col_name], errors='coerce')

# Preprocess categorical columns
for col_name in CATEGORICAL_COLS:
    inference_pdf[col_name] = inference_pdf[col_name].fillna('')

In [0]:
inference_pdf.shape

Out[33]: (479567, 16)

In [0]:
model = model = joblib.load(MODEL_PATH)

inference_pdf['prediction'] = model.predict(inference_pdf)
inference_pdf['probability_positive'] = model.predict_proba(inference_pdf)[:, 1]
inference_pdf['Account_Id'] = inference_pdf['Account_Id'].astype(str)
#inference_pdf['Reference_Number'] = inference_pdf['Reference_Number'].astype(str)

inference_sdf = spark.createDataFrame(inference_pdf[['Account_Id','prediction','probability_positive','MODEL_END_DATE']])

In [0]:
window_spec = Window.orderBy(psf.col("probability_positive"))

snowflake_predictions = (
    inference_sdf
    #.withColumn('probability_positive')
    .withColumn('SCORE', psf.ntile(100).over(window_spec).cast('integer'))
    .withColumn('SCORE',psf.col('SCORE')-1)
    .withColumn('MODEL_ID', psf.lit(MODEL_ID))
    .withColumn('MODEL_DATE', psf.to_timestamp(psf.current_date()))
    .withColumn('MODEL_END_DATE', psf.to_timestamp(psf.col('MODEL_END_DATE')))
    .select(
        'Account_Id',
       # 'Reference_Number',
        'probability_positive',
        'MODEL_ID',
        'SCORE',
        'MODEL_DATE',
        'MODEL_END_DATE'
    )
    .distinct()
)

snowflake_predictions.display()
snowflake_predictions.groupby('SCORE').count().display()
print(snowflake_predictions.count())
print(snowflake_predictions.select('Account_Id').distinct().count())

Account_Id,probability_positive,MODEL_ID,SCORE,MODEL_DATE,MODEL_END_DATE
626086,6.964107742533088E-4,INACTIVITY_CHURNER-V1,0,2024-06-30T00:00:00.000+0000,2024-07-06T00:00:00.000+0000
2026591,7.56042602006346E-4,INACTIVITY_CHURNER-V1,0,2024-06-30T00:00:00.000+0000,2024-07-06T00:00:00.000+0000
1657891,7.568071014247835E-4,INACTIVITY_CHURNER-V1,0,2024-06-30T00:00:00.000+0000,2024-07-06T00:00:00.000+0000
1253293,7.919027120806277E-4,INACTIVITY_CHURNER-V1,0,2024-06-30T00:00:00.000+0000,2024-07-06T00:00:00.000+0000
1881724,8.078314713202417E-4,INACTIVITY_CHURNER-V1,0,2024-06-30T00:00:00.000+0000,2024-07-06T00:00:00.000+0000
360598,8.566325413994491E-4,INACTIVITY_CHURNER-V1,0,2024-06-30T00:00:00.000+0000,2024-07-06T00:00:00.000+0000
1568012,8.973701624199748E-4,INACTIVITY_CHURNER-V1,0,2024-06-30T00:00:00.000+0000,2024-07-06T00:00:00.000+0000
1893668,9.16214135941118E-4,INACTIVITY_CHURNER-V1,0,2024-06-30T00:00:00.000+0000,2024-07-06T00:00:00.000+0000
1836564,9.176264284178615E-4,INACTIVITY_CHURNER-V1,0,2024-06-30T00:00:00.000+0000,2024-07-06T00:00:00.000+0000
1671336,9.295496274717152E-4,INACTIVITY_CHURNER-V1,0,2024-06-30T00:00:00.000+0000,2024-07-06T00:00:00.000+0000


SCORE,count
0,4796
1,4796
2,4796
3,4796
4,4796
5,4796
6,4796
7,4796
8,4796
9,4796


479567
479567


In [0]:
snowflake_predictions.write.option("overwriteSchema", "true").mode('overwrite').save(RAW_OUTPUT_TABLE)

In [0]:
# snowflake_predictions = spark.read.format("delta").load(RAW_OUTPUT_TABLE)

In [0]:
snowflake_predictions.write.format("snowflake").options(**options).option("dbtable", SNOWFLAKE_OUTPUT_TABLE).mode("append").save()